# Mermaid Diagram Generation

> Convert GraphContext objects to Mermaid.js diagram strings for visualization

In [ ]:
#| default_exp utils.mermaid

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict

from cjm_graph_plugin_system.core import GraphContext

## context_to_mermaid

Converts a `GraphContext` into a Mermaid.js flowchart diagram string. The resulting string can be rendered in Markdown environments that support Mermaid (GitHub, Jupyter with extensions, documentation sites, etc.).

In [ ]:
#| export
def context_to_mermaid(
    ctx: GraphContext,  # The GraphContext to visualize
    direction: str = "TD",  # Diagram direction: "TD" (top-down) or "LR" (left-right)
    node_color_map: Optional[Dict[str, str]] = None  # Map of node labels to CSS colors
) -> str:  # Mermaid.js diagram string
    """Convert a GraphContext into a Mermaid.js diagram string."""
    lines = [f"graph {direction}"]
    
    # Build node ID -> safe Mermaid ID mapping
    valid_ids = {n.id for n in ctx.nodes}
    id_to_mermaid = {n.id: f"N{i}" for i, n in enumerate(ctx.nodes)}
    
    # Generate node definitions
    for node in ctx.nodes:
        safe_id = id_to_mermaid[node.id]
        
        # Use 'name' or 'title' property for display, fallback to label
        display_text = node.properties.get('name', node.properties.get('title', node.label))
        display_text = str(display_text).replace('"', "'")
        
        lines.append(f'    {safe_id}["{display_text}"]')
        
        # Apply styling based on label
        if node_color_map and node.label in node_color_map:
            lines.append(f'    style {safe_id} fill:{node_color_map[node.label]}')
    
    # Generate edge definitions
    for edge in ctx.edges:
        if edge.source_id in valid_ids and edge.target_id in valid_ids:
            src_mermaid = id_to_mermaid[edge.source_id]
            tgt_mermaid = id_to_mermaid[edge.target_id]
            lines.append(f'    {src_mermaid} -->|{edge.relation_type}| {tgt_mermaid}')
    
    return "\n".join(lines)

In [ ]:
show_doc(context_to_mermaid)

---

### context_to_mermaid

```python

def context_to_mermaid(
    ctx:GraphContext, # The GraphContext to visualize
    direction:str='TD', # Diagram direction: "TD" (top-down) or "LR" (left-right)
    node_color_map:Optional=None, # Map of node labels to CSS colors
)->str: # Mermaid.js diagram string


```

*Convert a GraphContext into a Mermaid.js diagram string.*

### Example Usage

In [ ]:
import uuid
from cjm_graph_plugin_system.core import GraphNode, GraphEdge, GraphContext

# Create sample nodes
alice_id = str(uuid.uuid4())
bob_id = str(uuid.uuid4())
ml_id = str(uuid.uuid4())

nodes = [
    GraphNode(id=alice_id, label="Person", properties={"name": "Alice"}),
    GraphNode(id=bob_id, label="Person", properties={"name": "Bob"}),
    GraphNode(id=ml_id, label="Concept", properties={"name": "Machine Learning"})
]

# Create sample edges
edges = [
    GraphEdge(id=str(uuid.uuid4()), source_id=alice_id, target_id=ml_id, relation_type="MENTIONS"),
    GraphEdge(id=str(uuid.uuid4()), source_id=bob_id, target_id=ml_id, relation_type="MENTIONS"),
    GraphEdge(id=str(uuid.uuid4()), source_id=alice_id, target_id=bob_id, relation_type="KNOWS")
]

# Create GraphContext
ctx = GraphContext(nodes=nodes, edges=edges)

# Convert to Mermaid
mermaid_str = context_to_mermaid(ctx)
print(mermaid_str)

graph TD
    N0["Alice"]
    N1["Bob"]
    N2["Machine Learning"]
    N0 -->|MENTIONS| N2
    N1 -->|MENTIONS| N2
    N0 -->|KNOWS| N1


In [ ]:
# Test with node color mapping
color_map = {
    "Person": "#ffaaaa",
    "Concept": "#aaaaff"
}

mermaid_styled = context_to_mermaid(ctx, node_color_map=color_map)
print(mermaid_styled)

graph TD
    N0["Alice"]
    style N0 fill:#ffaaaa
    N1["Bob"]
    style N1 fill:#ffaaaa
    N2["Machine Learning"]
    style N2 fill:#aaaaff
    N0 -->|MENTIONS| N2
    N1 -->|MENTIONS| N2
    N0 -->|KNOWS| N1


In [ ]:
# Test left-right direction
mermaid_lr = context_to_mermaid(ctx, direction="LR")
print(mermaid_lr)

graph LR
    N0["Alice"]
    N1["Bob"]
    N2["Machine Learning"]
    N0 -->|MENTIONS| N2
    N1 -->|MENTIONS| N2
    N0 -->|KNOWS| N1


In [ ]:
# Test with empty context
empty_ctx = GraphContext(nodes=[], edges=[])
mermaid_empty = context_to_mermaid(empty_ctx)
print(mermaid_empty)
assert mermaid_empty == "graph TD"

graph TD


In [ ]:
# Test node with special characters in name
special_node = GraphNode(
    id=str(uuid.uuid4()),
    label="Quote",
    properties={"name": 'He said "hello"'}
)
special_ctx = GraphContext(nodes=[special_node], edges=[])
mermaid_special = context_to_mermaid(special_ctx)
print(mermaid_special)
assert "He said 'hello'" in mermaid_special  # Double quotes replaced with single quotes

graph TD
    N0["He said 'hello'"]


In [ ]:
# Test fallback to label when no name/title property
label_only_node = GraphNode(
    id=str(uuid.uuid4()),
    label="UnnamedThing",
    properties={"other": "value"}
)
label_ctx = GraphContext(nodes=[label_only_node], edges=[])
mermaid_label = context_to_mermaid(label_ctx)
print(mermaid_label)
assert "UnnamedThing" in mermaid_label

graph TD
    N0["UnnamedThing"]


In [ ]:
# Test that edges referencing missing nodes are skipped
orphan_edge = GraphEdge(
    id=str(uuid.uuid4()),
    source_id="nonexistent-1",
    target_id="nonexistent-2",
    relation_type="BROKEN"
)
single_node = GraphNode(id=str(uuid.uuid4()), label="Lonely", properties={"name": "Solo"})
orphan_ctx = GraphContext(nodes=[single_node], edges=[orphan_edge])
mermaid_orphan = context_to_mermaid(orphan_ctx)
print(mermaid_orphan)
assert "BROKEN" not in mermaid_orphan  # Orphan edge should not appear

graph TD
    N0["Solo"]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()